In [2]:
from glob import glob
import os
import json

json_file_list = [
 '/data-87-03/yuyue/datasets/wavcaps/json_files/BBC_Sound_Effects/bbc_final.json',
 '/data-87-03/yuyue/datasets/wavcaps/json_files/SoundBible/sb_final.json',
 '/data-87-03/yuyue/datasets/wavcaps/json_files/AudioSet_SL_flac/as_final.json',
 '/data-87-03/yuyue/datasets/wavcaps/json_files/FreeSound/fsd_final.json',
#  '/data-87-03/yuyue/datasets/wavcaps/json_files/blacklist/blacklist_exclude_test_ac.json',
#  '/data-87-03/yuyue/datasets/wavcaps/json_files/blacklist/blacklist_exclude_ub8k_esc50_vggsound.json',
#  '/data-87-03/yuyue/datasets/wavcaps/json_files/blacklist/blacklist_exclude_all_ac.json',
]

second_thread = 11
re = dict()
for json_file in json_file_list:
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    for info in data['data']:
        
        idx = info["id"][:-4] if info["id"].endswith('.wav') else info["id"]
        flac_path = f'/data-87-03/yuyue/datasets/wavcaps/wav_data/{json_file.split("/")[-2]}/{idx}.flac'
        if not os.path.exists(flac_path) or info['duration'] > second_thread:
            # print(flac_path)
            continue
        # d['audio_path'] = flac_path
        re[flac_path] = {"audio_caption": info['caption']}
    
len(re), re[flac_path]

(208900,
 {'audio_caption': 'Shaking a bean bag to sound like walking in tall grass.'})

In [ ]:
import pandas as pd
import csv
from glob import glob

# Read train.csv line by line
file_to_path = dict()

for file_path in glob('/data-04/xihua/data/audiocaps/audiocaps_haohe/audioset/zip_audios/*/*/*.wav'):
    file = file_path.split('/')[-1][1:-4]
    file_to_path[file] = file_path

# re = dict()
# save_path = '/home/chengxin/chengxin/vagen/data/tta/test_freesound_audiocaps.json'
with open('/data-04/xihua/data/audiocaps/audiocaps_haohe/metadata/audiocaps/train.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for line in csv_reader:
        d = dict()
        if line[1] not in file_to_path:
            print(line[1], end=" ")
            continue
        # d['audio_path'] = file_to_path[line[1]]
        file_name = file_to_path[line[1]]
        re[file_name] = {"audio_caption": line[3]}
        # d['audio_caption'] = line[3]
        # re.append(d)

len(re)

youtube_id 9T1QT2OQ0iM 90uYgzfHZxo 10RQxUIBTBU 6uOzzr5WPPU Yq7AzNoByHQ cnWZjejdwCM 2DsCAp68L18 1LY9j6EYAd8 bKbsPXcVBAI Jd5gy8IkI6A 5BUqA6zPREQ 6SujCvS39ig hZeqpirMQIA FWwFkZR4IqE AyPqm-tm2y4 H3znbPaPfL4 5U3YYrJabIQ -dg95jfWnP4 eWyg3HzR_Qo BVyAru1FDSg Mi9QA7NqRNk 3FwzJciQWq0 97B8WF-AHlg 1Ruw8hFefO0 NGPBlFWUrX0 3BOM4ZCP9f0 kZkmHTbS-Lc -nKy0zN4KVQ -rmnuALwnG0 YmcngI0QyvM GfOU0rW09Wg XE9XTkYD1pY 5XnjpgURugA A-ilwnVdZe4 A1LI2sGliaI 1IKMdBm4p0M 9wG9FrZN-8E 00HwP4YGQC4 2rhwJZ0cy3Y 3zmjqc-csQQ 2-2V3XHpBM8 fKJkguUPQ-M -uNTLpkNGHk BEkIKLVMAgA 8vnb6y7xRlI 8vrGEY3N_2U 4r6EpTvwZKs 6BpUKxf4c0Y 4PtuDVVvu-c 7TFH3ih6gbM -PK81pz3534 0xjrw60KFh0 OpGOdOjYwyk gYBCXXBAVoE 1RlJ7dvh5uQ 25BKXaN9y8c AhFnIwiRDuE SjjR7qq7P9g 9uvNvu3s2z8 MbO3Dkew4iM TNn0Qm6uh_o 6LvWwldXjzU r_F5gppUzRo 7xulC0Oa644 7ihcxu0eyVM 3pAJ8yjmC-U 317iLa-c9tc wKlMz_Sn2NM 5KiysMQvFDQ 9DJCYjwNIII 9NlRI1h14TQ -oVAWScjtJU CHjclKFVL7A _D_7Jip2TrI 1ylvtUqXkF4 kwcjvoqsu_M 61-cqnM_AGo 6VglYA9sUZc 7MTZW-2MayY 8pbpzdwlM90 B80W95e55NM 0qeurr_yeKc 5JRiL

258402

In [4]:
import json

save_path = '/home/chengxin/chengxin/vagen/data/tta/train_wavcaps_audiocaps_d10.json'

with open(save_path, 'w', encoding='utf-8') as file:
    json.dump(re, file, ensure_ascii=False, indent=4)
re[file_to_path[line[1]]], file_to_path[line[1]]

({'audio_caption': 'Humming with a distant train horn blowing'},
 '/data-04/xihua/data/audiocaps/audiocaps_haohe/audioset/zip_audios/unbalanced_train_segments/unbalanced_train_segments_part15/YHC-utKUX3a4.wav')

In [16]:
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
import json


pipe = StableAudioPipeline.from_pretrained("/home/chengxin/chengxin/vagen/assets/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# define the prompts
prompt = "The sound of a hammer hitting a wooden surface."
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda").manual_seed(0)


with open('/home/chengxin/chengxin/vagen/data/avsync/avsync-test-72B-captions.json', 'r') as f:
    data = json.load(f)

for path, info in data.items():
    audio = pipe(
        info['audio_caption'],
        negative_prompt=negative_prompt,
        num_inference_steps=200,
        audio_end_in_s=5.0,
        num_waveforms_per_prompt=3,
        generator=generator,
    ).audios

    output = audio[0].T.float().cpu().numpy()
    sf.write(f"/home/chengxin/chengxin/Dataset_Sound/VGGSound/generated_audios/stableaudio/avsync/{path.split('/')[-1].replace('.mp4', '.wav')}", output, pipe.vae.sampling_rate)

100%|██████████| 200/200 [00:15<00:00, 13.18it/s]


In [ ]:
import json, os

read_path = '/home/chengxin/chengxin/vagen/data/tta/train_avsync.json'
save_path = '/home/chengxin/chengxin/vagen/data/tta/train_avsync.json'
with open(read_path, 'r') as file:
    data = json.load(file)

i = 0
re = dict()
for path, info in data.items():
    info['audio_caption'] = info['label'].replace('_', " ")
    re[path] = info
#     path = f'/home/chengxin/chengxin/Dataset_Sound/VGGSound/dataset/test/10/{name}'
#     if not os.path.exists(path):
#         i += 1
#     re[path] = info

print(len(re), i)
with open(save_path, 'w') as file:
    json.dump(re, file, indent=4)

1271 0


: 

: 